In [2]:
import pandas as pd

In [3]:
data = pd.read_excel('D:\work\email_anay\服务单日志.xlsx')
data.head(3)

,log_data_id,log_type_id,main_relation_id,sub_relation_id,user_id,time,info
0,34215831804,4,360457518,0,10000,2025-01-01 00:01:42,[RecvEmailTask]新增服务单：[类型：售后]
1,34215902947,4,360457518,0,12090,2025-01-01 13:32:32,[email]修改服务单类型：售后 -> 售中-物流咨询
2,34215831855,4,360457519,0,10000,2025-01-01 00:01:50,[RecvEmailTask]新增服务单：[类型：售后]


In [4]:
service_num = len(data['main_relation_id'].unique())
print('未经过滤，所选时间段内的所有数据：')
print('总共进行了{}次类型修改操作'.format(len(data)))
print('总共有{}个服务单'.format(service_num))

未经过滤，所选时间段内的所有数据：
总共进行了111078次类型修改操作
总共有54984个服务单


In [10]:
# 把info列中包含"新增服务单"的行筛选出来
# a_data: 在所选时间段中新增的服务单
a_data = data[data['info'].str.contains('新增服务单')]
print("筛选出所选时间段内新增的服务单: ")
print('服务单数量:{}个'.format(len(a_data)))
a = a_data['main_relation_id'].unique()
a = set(a)
filtered_data = data[data['main_relation_id'].isin(a)]

print("服务单状态生成（修改）操作: {}次".format(len(filtered_data)))

筛选出所选时间段内新增的服务单: 
服务单数量:54152个
服务单状态生成（修改）操作: 109992次


In [16]:
# 把有且仅有两次操作的服务单筛选出来
b_data = filtered_data.groupby('main_relation_id').filter(lambda x: len(x) == 2)
sorted_data = b_data.sort_values(['main_relation_id', 'time'])
result_df = sorted_data.groupby('main_relation_id').agg({
    'info': lambda x: list(x)[:2]
}).reset_index()

# result_df['auto_type'] = result_df['info'].apply(lambda x: x[0] if len(x) > 0 else None)
result_df['auto_type'] = result_df['info'].apply(lambda x: x[0] if len(x) > 0 else None).str.extract(r'类型：([^， \]]+)')
result_df['modified_type'] = result_df['info'].apply(lambda x: x[1] if len(x) > 1 else None).str.extract(r'-> ([^， \]]+)')

final_df = result_df.rename(columns={'main_relation_id': 'service_id'}).drop('info', axis=1)

final_df.head(3)

,service_id,auto_type,modified_type
0,360457518,售后,售中-物流咨询
1,360457519,售后,售中-物流咨询
2,360457521,售后,售中-物流咨询


In [17]:
# 去掉modified_type列中是三级分类的行 （有两个"-"）
final_df = final_df[final_df['modified_type'].str.count('-') != 2]

In [15]:
sorted_data.to_excel('D:\work\email_anay\sorted_data.xlsx', index=False)

In [18]:
final_df.to_excel('D:\work\email_anay\服务单日志_处理后.xlsx', index=False)

In [ ]:
spu_id_list = a_data['main_relation_id'].to_list()
with open('D:\work\email_anay\email_service_ids.txt', 'w') as f:
    f.writelines(f"{i}\n" for i in spu_id_list)

In [ ]:
wrong_data = filtered_data.groupby('main_relation_id').filter(lambda x: len(x) == 2)
print("将时间段内有且仅有两次修改操作的服务单视作第一次自动分类出错")
print('自动分类出错的服务单个数：{}'.format(len(wrong_data['main_relation_id'].unique())))
print('错误率为{:.2f}%'.format(len(wrong_data['main_relation_id'].unique())/len(a_data)*100))    

In [ ]:
wrong_data.head(3)

### 统计第一次自动分类（错误分类）的各个类别占比

In [ ]:
a_data.loc[:, 'type'] = a_data['info'].str.extract(r'类型：([^， \]]+)')
a_data.head(3)

In [ ]:
first_wrong_data = wrong_data[wrong_data['info'].str.contains('新增服务单')].copy()
first_wrong_data.loc[:, 'type'] = first_wrong_data['info'].str.extract(r'类型：([^， \]]+)')

In [ ]:
first_wrong_data = first_wrong_data[['main_relation_id', 'time', 'type']]
first_wrong_data.head(3)
# 统计每种类型的错误数量
wrong_type = first_wrong_data.groupby('type').size()
wrong_type = wrong_type.sort_values(ascending=False)
print('错误类型统计(从高到低）：')
print(wrong_type)
print()
print('====================')
# 统计每种类型的错误率
type_num = a_data['type'].value_counts()
wrong_rate = (wrong_type / type_num * 100).round(2)
wrong_rate = wrong_rate.sort_values(ascending=False)
print('错误率统计：')
print('（该类型错误数量/该类型服务单数量）')
print()
print(wrong_rate.apply(lambda x: f'{x}%'))
